In [2]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask
import avg_utils as au
from regridding import regrid
from utils import loca_gard_mapping, city_list, trend_metric_ids

In [3]:
################
#### Paths #####
################
# Update these for reproduction

project_data_path = "/storage/group/pches/default/users/dcl5300/conus_comparison_lafferty-etal-2024"
project_code_path = "/storage/home/dcl5300/work/current_projects/conus_comparison_lafferty-etal-2024"

In [3]:
############
### Dask ###
############
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    # account="pches",
    account="open",
    cores=1,
    memory="10GiB",
    walltime="00:30:00",
)

cluster.scale(jobs=30)  # ask for jobs

from dask.distributed import Client

client = Client(cluster)

client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.8.221:39699,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Calculate averages

In [11]:
# Fit all
proj_year_groups = [[2040,2060], [2080,2100]]
hist_year_groups = [None, [1950,2014]]

for metric_id in trend_metric_ids:
    for proj_years in proj_year_groups:
        for hist_years in hist_year_groups:
            au.avg_calc_all(metric_id = metric_id,
                            proj_years = proj_years,
                            hist_years = hist_years)

## Regridding

In [4]:
%%time
# Regrid to LOCA using NN
method = "nearest"
target = "LOCA2"

for metric_id in trend_metric_ids:
    # Save path
    store_path = f"{project_data_path}/averages/loca_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/averages/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/averages/original_grid/{metric_id}/GARD-LENS*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
                del ds_out, ds_tmp
            except Exception as e:
                print(file)
                print(e)

CPU times: user 2min 33s, sys: 19.9 s, total: 2min 53s
Wall time: 8min 17s


In [6]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "GARD-LENS"

for metric_id in trend_metric_ids:
    # Save path
    store_path = f"{project_data_path}/averages/gard_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/averages/original_grid/{metric_id}/STAR-ESDM*") + \
            glob(f"{project_data_path}/averages/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
                del ds_out, ds_tmp
            except Exception as e:
                print(file)
                print(e)

CPU times: user 2min 51s, sys: 19.8 s, total: 3min 11s
Wall time: 9min 27s


In [7]:
%%time
# Regrid to GARD-LENS using NN
method = "nearest"
target = "STAR-ESDM"

for metric_id in trend_metric_ids:
    # Save path
    store_path = f"{project_data_path}/averages/star_grid/{metric_id}"
    # Get all files
    files = glob(f"{project_data_path}/averages/original_grid/{metric_id}/GARD-LENS*") + \
            glob(f"{project_data_path}/averages/original_grid/{metric_id}/LOCA2*")

    # Loop through
    for file in files:
        # Check if done
        file_name = file.split('/')[-1]
        store_name = file_name.replace(".nc", f"_{method}.nc")
        if not os.path.exists(f"{store_path}/{store_name}"):
            try:
                # Regrid and store
                ds_tmp = xr.open_dataset(file)
                ds_out = regrid(ds_tmp, method=method, target=target)
                ds_out.to_netcdf(f"{store_path}/{store_name}")
                del ds_out, ds_tmp
            except Exception as e:
                print(file)
                print(e)

CPU times: user 4min 31s, sys: 54.7 s, total: 5min 25s
Wall time: 17min 39s
